In [39]:
import scipy as scipy
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt 
import random as rand
import multiprocessing

from joblib import Parallel, delayed
from scipy.stats import genextreme as gev
from netCDF4 import Dataset


# Read in data
Read lat and lon from CaMa file

Read saved Matlab arrays of maximum annual outflow 

In [3]:
CaMaFile = Dataset('/Users/meganfowler/gp_fuse/Flooding-physiology/MatlabData/fldare1161.nc')
# print(CaMaFile.dimensions.keys())

lat =  CaMaFile.variables['lat'][:] 
lon = CaMaFile.variables['lon'][:] 

In [4]:
#matData   = sio.loadmat('/Users/meganfowler/Documents/MATLAB/FloodProject/CaMaFlood/Physiology/30yr/control_outflw.mat')

matData   = sio.loadmat('/Users/meganfowler/Documents/MATLAB/FloodProject/CaMaFlood/Physiology/30yr/control_outflw.mat')
maxYearly = matData['maxYearly_full']

fullData   = sio.loadmat('/Users/meganfowler/Documents/MATLAB/FloodProject/CaMaFlood/Physiology/30yr/full_outflw.mat')
fullYearly = fullData['maxYearly_full']

physData = sio.loadmat('/Users/meganfowler/Documents/MATLAB/FloodProject/CaMaFlood/Physiology/30yr/physiology_outflw.mat')
physYearly = physData['maxYearly_full']

radData = sio.loadmat('/Users/meganfowler/Documents/MATLAB/FloodProject/CaMaFlood/Physiology/30yr/radiation_outflw.mat')
radYearly = radData['maxYearly_full']



In [35]:
#Specify return period and number of bootstraps
retPeriod = 100.0
nsamples = 1 

#Define empty arrays
Tfull = np.empty([len(lon),len(lat)])
Tphys = np.empty([len(lon),len(lat)])
Trad  = np.empty([len(lon),len(lat)])


In [ ]:
# TEST 
iRand = np.random.randint(0,30,size=30)
rec     = maxYearly[:,:,iRand]
recFull = fullYearly[:,:,iRand]
recPhys = physYearly[:,:,iRand]
recRad  = radYearly[:,:,iRand]

num_cores = multiprocessing.cpu_count()
print(num_cores)
#test = Parallel(n_jobs=num_cores)(delayed(computeGEVbootstrap)(rec,recFull,recPhys,recRad,i) for i in range(len(lon)))

test = Parallel(n_jobs=num_cores)(delayed(computeGEVbootstrap)(rec,recFull,recPhys,recRad,i) for i in range(100))



In [ ]:
def computeGEVbootstrap(recArray,recFullArr,recPhysArr,recRadArr,i):
    print(i) 
    
    #print(len(lat))
    
    #for j in range(len(lat)):   

    for j in range(100):   

        #print(j)
        rec     = recArray[i,j,:]
        recFull = recFullArr[i,j,:]
        recPhys = recPhysArr[i,j,:]
        recRad  = recRadArr[i,j,:]
       
        # Check for NaN 
        validCount = np.count_nonzero(~np.isnan(rec))
              
        if validCount>5: 
            #print("Starting to compute...")
            
            gev_fit = gev.fit(rec)
            K = gev_fit[0]
            M = gev_fit[1]
            S = gev_fit[2]
    
            #Compute discharge of 100-year flood 
            R100 = gev.ppf((1-(1/retPeriod)),K,loc=M,scale=S)
    
            #Compute fit for CO2 experiments
            gevFull = gev.fit(recFull)
            gevPhys = gev.fit(recPhys)
            gevRad  = gev.fit(recRad)
    
            cdfFull = gev.cdf(R100,*gevFull)    #Compute CDF at the point of R100 
            cdfPhys = gev.cdf(R100,*gevPhys)
            cdfRad  = gev.cdf(R100,*gevRad)
    
            Tfull[i,j] = 1/(1-cdfFull)    #Return Period 
            Tphys[i,j] = 1/(1-cdfPhys)
            Trad[i,j]= 1/(1-cdfRad)
            
            #print("Finished computing GEV...")
            
        #print("HITHIT")
                
    #print(i)
    
    return {'Tfull':Tfull, 'Tphys':Tphys ,'Trad':Trad}

In [ ]:

b=0
while b<=nsamples
    #i=0
    
    iRand = np.random.randint(0,30,size=30)
    
    rec     = maxYearly[:,:,iRand]
    recFull = fullYearly[:,:,iRand]
    recPhys = physYearly[:,:,iRand]
    recRad  = radYearly[:,:,iRand]
    
    bootstrap[b,:,:,:] = Parallel(n_jobs=10)(delayed(computeGEVbootstrap)(rec,recFull,recPhys,recRad,i) for i in range(len(lon)))
    
    #for i in range(0, len(lon)):
    #while i<len(lon):
        #j=0
        #while j<len(lat):
        
            # Compute GEV fit for CTRL and compute discharge of "retPeriod"-year flood

       
            # Check for NaN 
            #validCount = np.count_nonzero(~np.isnan(rec))
        
            #if validCount>5:   
                
                #bootstrap[b,i,j,:] = Parallel(n_jobs=10)(delayed(computeGEVbootstrap)(rec,recFull,recPhys,recRad) for i in inputs)

                
                
        
                #gev_fit = gev.fit(rec)
        
                #K = gev_fit[0]
                #M = gev_fit[1]
                #S = gev_fit[2]
        
                #R100 = gev.ppf((1-(1/retPeriod)),K,loc=M,scale=S)
        
                # Compute return period of that magnitude flood in 4xCO2 experiments 
        
                #gevFull = gev.fit(recFull)   #Fit GEV to get parameters in each case 
                #gevPhys = gev.fit(recPhys)
                #gevRad  = gev.fit(recRad)       
        
                #cdfFull = gev.cdf(R100,*gevFull)    #Compute CDF at the point of R100 
                #cdfPhys = gev.cdf(R100,*gevPhys)
                #cdfRad  = gev.cdf(R100,*gevRad)
        
                #Tfull_boot[b,i,j] = 1/(1-cdfFull)    #Return Period 
                #Tphys_boot[b,i,j] = 1/(1-cdfPhys)
                #Trad_boot[b,i,j] = 1/(1-cdfRad)
            
            #j = j+1
        #i = i+1 
    b = b+1

